In [1]:
import pandas as pd  
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()

url = 'https://finance.vietstock.vn/chi-so-nganh.htm'
driver.get(url)

try:
    login_link = driver.find_element(By.XPATH, '//a[@class="title-link btnlogin-link"]')
    login_link.click()

    username_input = driver.find_element(By.XPATH, '//*[@id="txtEmailLogin"]')
    password_input = driver.find_element(By.XPATH, '//*[@id="txtPassword"]')

    username = "doankhoa126@gmail.com"
    password = "wolf2952001"

    username_input.send_keys(username)
    password_input.send_keys(password)

    login_button = driver.find_element(By.XPATH, '//*[@id="btnLoginAccount"]')
    login_button.click()

    time.sleep(5)

    data = [] 

    for loop_count in range(1):
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'table.table-striped.table-hover')))


        rows = table.find_elements(By.TAG_NAME, 'tr')

        for row in rows:
            cells = row.find_elements(By.TAG_NAME, 'td')
            row_data = [cell.text for cell in cells]
            data.append(row_data)


    df = pd.DataFrame(data)
  

    print(df)

except Exception as e:
    print(f"Lỗi: {str(e)}")
finally:
    driver.quit()


       0                                   1                    2           3  \
0   None                                None                 None        None   
1      1                            Bán buôn     31.68 ( +1.27 %)  15/11/2023   
2      2                            Bảo hiểm    119.75 ( +0.17 %)  15/11/2023   
3      3                        Bất động sản    403.62 ( +0.91 %)  15/11/2023   
4      4                         Chứng khoán    140.50 ( +0.78 %)  15/11/2023   
5      5              Công nghệ và thông tin    414.53 ( -0.15 %)  15/11/2023   
6      6                              Bán lẻ    604.55 ( +0.62 %)  15/11/2023   
7      7                   Chăm sóc sức khỏe    464.70 ( +0.48 %)  15/11/2023   
8      8                         Khai khoáng    232.76 ( -0.61 %)  15/11/2023   
9      9                           Ngân hàng    614.71 ( +1.50 %)  15/11/2023   
10    10                    Nông - Lâm - Ngư     27.12 ( +0.23 %)  15/11/2023   
11    11                SX T

In [2]:
import pandas as pd  
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pyodbc

In [3]:
def trading_Sector():
    driver = webdriver.Chrome()

    url = 'https://finance.vietstock.vn/chi-so-nganh.htm'
    driver.get(url)

    try:
        login_link = driver.find_element(By.XPATH, '//a[@class="title-link btnlogin-link"]')
        login_link.click()

        username_input = driver.find_element(By.XPATH, '//*[@id="txtEmailLogin"]')
        password_input = driver.find_element(By.XPATH, '//*[@id="txtPassword"]')

        username = "doankhoa126@gmail.com"
        password = "wolf2952001"

        username_input.send_keys(username)
        password_input.send_keys(password)

        login_button = driver.find_element(By.XPATH, '//*[@id="btnLoginAccount"]')
        login_button.click()

        time.sleep(5)

        data = [] 

        for loop_count in range(1):
            table = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'table.table-striped.table-hover')))


            rows = table.find_elements(By.TAG_NAME, 'tr')

            for row in rows:
                cells = row.find_elements(By.TAG_NAME, 'td')
                row_data = [cell.text for cell in cells]
                data.append(row_data)


        df = pd.DataFrame(data)

    except Exception as e:
        print(f"Lỗi: {str(e)}")
    finally:
        driver.quit()
        
    df.columns = ['stt', 'sector', 'change', 'date', 'Vol', 'Val', 'foreignBuy', 'foreignSell']
    df = df.drop(0)
    
    df['changeVal'] = df['change'].str.extract(r'([\d.]+)')[0]
    return df

In [4]:
def insert_Trading(df):
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=MSI\\SQLEXPRESS;'
                          'Database=dataFinance;'
                          'Trusted_Connection=yes;')


    cursor = conn.cursor()

    df = df.replace('-', 0)

    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')





    def custom_float_conversion(value):
        if isinstance(value, str):
            try:
                return float(value.replace(',', ''))
            except ValueError:
                return value
        elif isinstance(value, int):
            return float(value)
        else:
            return value


    columns_to_convert_to_float = ['Vol','Val','foreignBuy','foreignSell','changeVal']

    for column in columns_to_convert_to_float:
        df[column] = df[column].apply(custom_float_conversion)


    for index, row in df.iterrows():
    # Chuyển đổi các giá trị rỗng (dấu '-') thành 0
        for col in df.columns:
            if pd.isna(row[col]) or row[col] == '-':
                row[col] = 0

    for index, row in df.iterrows():
        cursor.execute("INSERT INTO tradingIndustry (sector,change,date,Vol,Val,foreignBuy,foreignSell,changeVal) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
                       row['sector'], row['change'], row['date'], row['Vol'], row['Val'], row['foreignBuy'], row['foreignSell'], row['changeVal'])

        
    

    conn.commit()
    conn.close()


In [5]:
def main():
    df = trading_Sector()
    
    insert_Trading(df)
    
    print("insert success tranding Sector")

if __name__ == '__main__':
    main()

insert success tranding Sector
